This is a problem of building five houses in different locations; the masonry, roofing, painting, etc. must be scheduled. Some tasks must necessarily take place before others and these requirements are expressed through precedence constraints.

There are three workers, and each worker has a given skill level for each task. Each task requires one worker; the worker assigned must have a non-null skill level for the task. A worker can be assigned to only one task at a time.

The objective of this problem is to maximize the skill level used for all the tasks.

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/cp/jupyter/house_building.ipynb

https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/cp/basic/house_building.py


In [1]:
from docplex.cp.model import CpoModel
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 35, 10
import docplex.cp.utils_visu as visu
from docplex.cp.expression import transition_matrix

# Data Creation

In [ ]:
inputs = {}

Houses = pd.DataFrame([])
Houses['house']=[str(i) for i in range(1,6)]


Tasks = pd.DataFrame(\
    [   ["masonry",   35],
         ["carpentry", 15],
         ["plumbing",  40],
         ["ceiling",   15],
         ["roofing",    5],
         ["painting",  10],
         ["windows",    5],
         ["facade",    10],
         ["garden",     5],
         ["moving",     5]], columns = ["task", "duration"])



TaskPrecedence = pd.DataFrame\
                  ([["masonry",   "carpentry"],
                    ["masonry",   "plumbing"],
                    ["masonry",   "ceiling"],
                    ["carpentry", "roofing"],
                    ["ceiling",   "painting"],
                    ["roofing",   "windows"],
                    ["roofing",   "facade"],
                    ["plumbing",  "facade"],
                    ["roofing",   "garden"],
                    ["plumbing",  "garden"],
                    ["windows",   "moving"],
                    ["facade",    "moving"],
                    ["garden",    "moving"],
                    ["painting",  "moving"]], columns = ["beforeTask", "afterTask"])


Skills  =  pd.DataFrame\
        ([["Joe",  "masonry",   9],
          ["Joe",  "carpentry", 7],
          ["Joe",  "ceiling",   5],
          ["Joe",  "roofing",   6],
          ["Joe",  "windows",   8],
          ["Joe",  "facade",    5],
          ["Joe",  "garden",    5],
          ["Joe",  "moving",    6],
          ["Jack", "masonry",   5],
          ["Jack", "plumbing",  7],
          ["Jack", "ceiling",   8],
          ["Jack", "roofing",   7],
          ["Jack", "painting",  9],
          ["Jack", "facade",    5],
          ["Jack", "garden",    5],
          ["Jim",  "carpentry", 5],
          ["Jim",  "painting",  6],
          ["Jim",  "windows",   5],
          ["Jim",  "garden",    9],
          ["Jim",  "moving",    8]], columns = ["worker", "task", "level"] )
          

# Worker and continuity requirements: if the Task 1 is done on the house, he must do the task 2 in this house
Continuities = pd.DataFrame([
                ["Joe",  "masonry",   "carpentry"],
                ["Jack", "roofing",   "facade"],
                ["Joe",  "carpentry", "roofing"],
                ["Jim",  "garden",    "moving"]], columns = ["worker", "task1", "task2"])


inputs['Houses']= Houses
inputs['Tasks']= Tasks
inputs['TaskPrecedence']= TaskPrecedence
inputs['Skills']= Skills
inputs['Continuities']= Continuities

inputs.keys()

# Model Building

create model

In [ ]:
mdl = CpoModel(name="HouseBuilding")

MAX_SCHEDULE = Tasks['duration'].sum()*5

Define Decision variables

In [ ]:
# table of interval variable for each house and task
tasks = pd.merge(Houses, Tasks, how = 'cross')
tasks['TaskDV']=0
for index, row in tasks.iterrows():
    dv =  mdl.interval_var((0, MAX_SCHEDULE) , (0, MAX_SCHEDULE) , size=row.duration, name="house:{} task:{}".format(row.house, row.task))
    tasks.at[index, 'TaskDV']=dv

display(tasks.head())



#table of interval variable for each house and skill
wtasks = pd.merge(Houses, Skills, how = 'cross')
wtasks['SkillDV']=0
for index, row in wtasks.iterrows():
    dv =  mdl.interval_var(optional = True , name="house:{} worker:{} skill:{}".format(row.house, row.worker, row.task))
    wtasks.at[index, 'SkillDV']=dv


display(wtasks.head())

Define Constraints

In [ ]:
# Temporal constraints.TASK_PRECEDENCES  

u = pd.merge(tasks,TaskPrecedence, left_on = 'task', right_on = 'beforeTask' )[['house', 'TaskDV', 'beforeTask','afterTask']].rename(columns = {'TaskDV':"TaskDV_before"})
v= pd.merge(u, tasks, left_on = ['afterTask','house'] , right_on = ['task','house']).rename(columns = {'TaskDV':"TaskDV_after"})[['house', 'TaskDV_before', 'beforeTask','afterTask','TaskDV_after']]
display(v.head())


for index, row in v.iterrows():
    mdl.add( mdl.end_before_start(row.TaskDV_before, row.TaskDV_after))


In [ ]:
# For each task in a house, only one of the workers should be assigned.

task_workers = pd.merge(tasks, wtasks, on = ['house','task'])
task_workers= task_workers.groupby(['house','task','duration']).agg(list)
task_workers['TaskDV'] = task_workers['TaskDV'].apply(lambda x:x[0])


display(task_workers.head())

for row in task_workers.itertuples():
    mdl.add(  mdl.alternative(row.TaskDV, row.SkillDV, cardinality=1)  )


In [ ]:
# continuity constraints

u = pd.merge(wtasks, Continuities, left_on = ['task', 'worker'], right_on = ['task1', 'worker'])[['house','worker','task1', 'task2','SkillDV']].\
    rename(columns ={'SkillDV':'SkillDV_1'} )

v= pd.merge(wtasks, u, left_on = ['house', 'task', 'worker'], right_on = ['house', 'task2', 'worker']).\
    rename(columns ={'SkillDV':'SkillDV_2'} )[['house','worker','task1', 'task2','SkillDV_1', 'SkillDV_2']].sort_values(['house','worker'])

display(v.head())

for row in v.itertuples(): 
    mdl.add ( mdl.presence_of ( row.SkillDV_1 )== mdl.presence_of(row.SkillDV_2))


In [ ]:
# No overlap constraint

workers_per_house = wtasks[['worker','SkillDV' ]].groupby(['worker']).agg(list)
display(workers_per_house.head(2))

for row in workers_per_house.itertuples():
    mdl.add ( mdl.no_overlap( row.SkillDV)) 

Objective function

In [ ]:
#wtasks['size']=wtasks['SkillDV'].apply(lambda x:mdl.end_of(x))
wtasks['SkillDV_presence']= wtasks['SkillDV'].apply(lambda x:mdl.presence_of(x))

#obj = mdl.minimize ( mdl.max ( wtasks['size']  * wtasks['SkillDV_presence']  ) )
obj = mdl.maximize ( mdl.sum ( wtasks['level'] *  wtasks['SkillDV_presence'])) 

mdl.add(obj)

# Solve

In [ ]:
print("\nSolving model....")
msol = mdl.solve(TimeLimit=120)

# Print solution
print("Solve status: " + msol.get_solve_status())

msol.get_objective_values()


In [ ]:
tasks['TaskDV_sol'] = tasks['TaskDV'].apply(lambda x:msol.get_var_solution(x))
tasks['TaskDV_sol_start']=tasks['TaskDV_sol'].apply(lambda x:x.get_start())
tasks['TaskDV_sol_end']=tasks['TaskDV_sol'].apply(lambda x:x.get_end())
tasks.sort_values(['TaskDV_sol_start','TaskDV_sol_end']).head()


In [ ]:
wtasks['SkillDV_sol'] = wtasks['SkillDV'].apply(lambda x:msol.get_var_solution(x))
wtasks['SkillDV_sol_start']=wtasks['SkillDV_sol'].apply(lambda x:x.get_start())
wtasks['SkillDV_sol_end']=wtasks['SkillDV_sol'].apply(lambda x:x.get_end())
wtasks[~pd.isnull(wtasks['SkillDV_sol_start'])].sort_values(['SkillDV_sol_start','SkillDV_sol_end']).head()

In [ ]:
u= pd.merge( wtasks[['house','worker','task', 'SkillDV_sol_start','SkillDV_sol_end']], 
         tasks[['house','task', 'TaskDV_sol_start','TaskDV_sol_end']], left_on=['house','task','SkillDV_sol_start','SkillDV_sol_end'], \
            right_on = ['house','task','TaskDV_sol_start','TaskDV_sol_end'], how = 'right')

u.sort_values(['TaskDV_sol_start','TaskDV_sol_end'])

In [ ]:
rcParams['figure.figsize'] = 30, 5
POP_UP_GRAPHIC=False

colors ={"masonry":'yellow',   "carpentry":'lightblue', "plumbing":  'lightgreen',"ceiling" :'cyan' ,"roofing":'linen',"painting":'green',"windows":'ivory',"facade":  'seashell',"garden":'gold',"moving":
 'pink'}

for i in tasks['TaskDV'].values:
    wt = msol.get_var_solution(i)  

    house = wt.get_name().split(" ")[0].split(":")[1]
    task = wt.get_name().split(" ")[1].split(":")[1]
    worker = u[ (u['house']==house) & (u['task']==task)]['worker'].values[0]


    visu.interval(wt, colors[task], ':'.join([house, task, worker])) 

visu.show()

# Case 2 :

source:https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/cp/jupyter/scheduling_tuto.ipynb

In [52]:
NbHouses = 4

#WorkerNames = pd.DataFrame(["Joe", "Jim"], columns = ['worker'])

Tasks = pd.DataFrame(["masonry", "carpentry", "plumbing", "ceiling", "roofing", "painting", "windows", "facade", "garden", "moving"], columns = ['TaskName'])
Tasks['Duration']= [35, 15, 40, 15, 5, 10, 5, 10, 5, 5]


Workers = pd.DataFrame.from_dict( {"masonry"  : "Joe" , 
          "carpentry": "Joe" , 
          "plumbing" : "Jim" , 
          "ceiling"  : "Jim" , 
          "roofing"  : "Joe" , 
          "painting" : "Jim" , 
          "windows"  : "Jim" , 
          "facade"   : "Joe" , 
          "garden"   : "Joe" , 
          "moving"   : "Jim"}, orient='index').reset_index(drop= False)

Workers.columns = ['TaskName','WorkerName' ]

Precedences = [("masonry", "carpentry"),("masonry", "plumbing"),
               ("masonry", "ceiling"), ("carpentry", "roofing"),
               ("ceiling", "painting"), ("roofing", "windows"),  
               ("roofing", "facade"), ("plumbing", "facade"),
               ("roofing", "garden"), ("plumbing", "garden"),
               ("windows", "moving"), ("facade", "moving"),  
               ("garden", "moving"), ("painting", "moving")]

Precedences = pd.DataFrame(Precedences, columns = ["BeforeTask", "AfterTask"])


Houses = pd.DataFrame([])
Houses['HouseName'] = range(NbHouses)
Houses['ReleaseDate'] = [  0,     0,   151,    59,   243][0:NbHouses]
Houses['DueDate']     = [120,   212,   304,   181,   425][0:NbHouses]
Houses['Weight']      = [100.0, 100.0, 100.0, 200.0, 100.0][0:NbHouses]

#The objective is to minimize the cost incurred through tardiness and length costs.


In [53]:
mdl = CpoModel()

INTERVAL_MAX = 1000

Houses['HouseDV']=0
for index, row in Houses.iterrows():
    dv =  mdl.interval_var(start = (int(row.ReleaseDate), INTERVAL_MAX ) , name="house:{}".format(int(row.HouseName)))
    Houses.at[index, 'HouseDV']=dv

display(Houses.head())

,HouseName,ReleaseDate,DueDate,Weight,HouseDV
0,0,0,120,100.0,"""house:0"" = intervalVar(start=0..1000)"
1,1,0,212,100.0,"""house:1"" = intervalVar(start=0..1000)"
2,2,151,304,100.0,"""house:2"" = intervalVar(start=151..1000)"
3,3,59,181,200.0,"""house:3"" = intervalVar(start=59..1000)"


In [54]:
house_tasks = pd.merge(Houses[['HouseName']], Tasks[['TaskName','Duration']], how = 'cross')

house_tasks ['TaskDV']=0
for index, row in house_tasks.iterrows():
    dv =  mdl.interval_var(size = row.Duration , name="house:{} task:{}".format(int(row.HouseName),row.TaskName))
    house_tasks.at[index, 'TaskDV']=dv

display(house_tasks.head())

,HouseName,TaskName,Duration,TaskDV
0,0,masonry,35,"""house:0 task:masonry"" = intervalVar(size=35)"
1,0,carpentry,15,"""house:0 task:carpentry"" = intervalVar(size=15)"
2,0,plumbing,40,"""house:0 task:plumbing"" = intervalVar(size=40)"
3,0,ceiling,15,"""house:0 task:ceiling"" = intervalVar(size=15)"
4,0,roofing,5,"""house:0 task:roofing"" = intervalVar(size=5)"


In [55]:
# Temporal constraints.TASK_PRECEDENCES  

u = pd.merge(house_tasks,Precedences, left_on = 'TaskName', right_on = 'BeforeTask' )[['HouseName', 'TaskDV', 'BeforeTask','AfterTask']].rename(columns = {'TaskDV':"TaskDV_before"})
v= pd.merge(u, house_tasks, left_on = ['AfterTask','HouseName'] , right_on = ['TaskName','HouseName']).rename(columns = {'TaskDV':"TaskDV_after"})[['HouseName', 'TaskDV_before', 'BeforeTask','AfterTask','TaskDV_after']]
display(v.head())


for index, row in v.iterrows():
    mdl.add( mdl.end_before_start(row.TaskDV_before, row.TaskDV_after))

,HouseName,TaskDV_before,BeforeTask,AfterTask,TaskDV_after
0,0,"""house:0 task:masonry"" = intervalVar(size=35)",masonry,carpentry,"""house:0 task:carpentry"" = intervalVar(size=15)"
1,0,"""house:0 task:masonry"" = intervalVar(size=35)",masonry,plumbing,"""house:0 task:plumbing"" = intervalVar(size=40)"
2,0,"""house:0 task:masonry"" = intervalVar(size=35)",masonry,ceiling,"""house:0 task:ceiling"" = intervalVar(size=15)"
3,1,"""house:1 task:masonry"" = intervalVar(size=35)",masonry,carpentry,"""house:1 task:carpentry"" = intervalVar(size=15)"
4,1,"""house:1 task:masonry"" = intervalVar(size=35)",masonry,plumbing,"""house:1 task:plumbing"" = intervalVar(size=40)"


In [56]:
# a house interval spans all its tasks

In [57]:
df = house_tasks[['HouseName','TaskDV']].groupby('HouseName').agg(list)
df = pd.merge(df, Houses[['HouseName','HouseDV']], on = 'HouseName')
display(df.head(1))

for index, row in df.iterrows():
    mdl.add( mdl.span( row.HouseDV, row. TaskDV) )

,HouseName,TaskDV,HouseDV
0,0,"[""house:0 task:masonry"" = intervalVar(size=35), ""house:0 task:carpentry"" = intervalVar(size=15), ""house:0 task:plumbing"" = intervalVar(size=40), ""house:0 task:ceiling"" = intervalVar(size=15), ""house:0 task:roofing"" = intervalVar(size=5), ""house:0 task:painting"" = intervalVar(size=10), ""house:0 task:windows"" = intervalVar(size=5), ""house:0 task:facade"" = intervalVar(size=10), ""house:0 task:garden"" = intervalVar(size=5), ""house:0 task:moving"" = intervalVar(size=5)]","""house:0"" = intervalVar(start=0..1000)"


In [58]:
df

,HouseName,TaskDV,HouseDV
0,0,"[""house:0 task:masonry"" = intervalVar(size=35), ""house:0 task:carpentry"" = intervalVar(size=15), ""house:0 task:plumbing"" = intervalVar(size=40), ""house:0 task:ceiling"" = intervalVar(size=15), ""house:0 task:roofing"" = intervalVar(size=5), ""house:0 task:painting"" = intervalVar(size=10), ""house:0 task:windows"" = intervalVar(size=5), ""house:0 task:facade"" = intervalVar(size=10), ""house:0 task:garden"" = intervalVar(size=5), ""house:0 task:moving"" = intervalVar(size=5)]","""house:0"" = intervalVar(start=0..1000)"
1,1,"[""house:1 task:masonry"" = intervalVar(size=35), ""house:1 task:carpentry"" = intervalVar(size=15), ""house:1 task:plumbing"" = intervalVar(size=40), ""house:1 task:ceiling"" = intervalVar(size=15), ""house:1 task:roofing"" = intervalVar(size=5), ""house:1 task:painting"" = intervalVar(size=10), ""house:1 task:windows"" = intervalVar(size=5), ""house:1 task:facade"" = intervalVar(size=10), ""house:1 task:garden"" = intervalVar(size=5), ""house:1 task:moving"" = intervalVar(size=5)]","""house:1"" = intervalVar(start=0..1000)"
2,2,"[""house:2 task:masonry"" = intervalVar(size=35), ""house:2 task:carpentry"" = intervalVar(size=15), ""house:2 task:plumbing"" = intervalVar(size=40), ""house:2 task:ceiling"" = intervalVar(size=15), ""house:2 task:roofing"" = intervalVar(size=5), ""house:2 task:painting"" = intervalVar(size=10), ""house:2 task:windows"" = intervalVar(size=5), ""house:2 task:facade"" = intervalVar(size=10), ""house:2 task:garden"" = intervalVar(size=5), ""house:2 task:moving"" = intervalVar(size=5)]","""house:2"" = intervalVar(start=151..1000)"
3,3,"[""house:3 task:masonry"" = intervalVar(size=35), ""house:3 task:carpentry"" = intervalVar(size=15), ""house:3 task:plumbing"" = intervalVar(size=40), ""house:3 task:ceiling"" = intervalVar(size=15), ""house:3 task:roofing"" = intervalVar(size=5), ""house:3 task:painting"" = intervalVar(size=10), ""house:3 task:windows"" = intervalVar(size=5), ""house:3 task:facade"" = intervalVar(size=10), ""house:3 task:garden"" = intervalVar(size=5), ""house:3 task:moving"" = intervalVar(size=5)]","""house:3"" = intervalVar(start=59..1000)"


In [59]:
df = pd.merge( house_tasks, Workers, on = ['TaskName'])
Worker_tasks = df[['HouseName','WorkerName','TaskDV']].groupby('WorkerName').agg(list).reset_index()
Worker_tasks

,WorkerName,HouseName,TaskDV
0,Jim,"[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]","[""house:0 task:plumbing"" = intervalVar(size=40), ""house:1 task:plumbing"" = intervalVar(size=40), ""house:2 task:plumbing"" = intervalVar(size=40), ""house:3 task:plumbing"" = intervalVar(size=40), ""house:0 task:ceiling"" = intervalVar(size=15), ""house:1 task:ceiling"" = intervalVar(size=15), ""house:2 task:ceiling"" = intervalVar(size=15), ""house:3 task:ceiling"" = intervalVar(size=15), ""house:0 task:painting"" = intervalVar(size=10), ""house:1 task:painting"" = intervalVar(size=10), ""house:2 task:painting"" = intervalVar(size=10), ""house:3 task:painting"" = intervalVar(size=10), ""house:0 task:windows"" = intervalVar(size=5), ""house:1 task:windows"" = intervalVar(size=5), ""house:2 task:windows"" = intervalVar(size=5), ""house:3 task:windows"" = intervalVar(size=5), ""house:0 task:moving"" = intervalVar(size=5), ""house:1 task:moving"" = intervalVar(size=5), ""house:2 task:moving"" = intervalVar(size=5), ""house:3 task:moving"" = intervalVar(size=5)]"
1,Joe,"[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]","[""house:0 task:masonry"" = intervalVar(size=35), ""house:1 task:masonry"" = intervalVar(size=35), ""house:2 task:masonry"" = intervalVar(size=35), ""house:3 task:masonry"" = intervalVar(size=35), ""house:0 task:carpentry"" = intervalVar(size=15), ""house:1 task:carpentry"" = intervalVar(size=15), ""house:2 task:carpentry"" = intervalVar(size=15), ""house:3 task:carpentry"" = intervalVar(size=15), ""house:0 task:roofing"" = intervalVar(size=5), ""house:1 task:roofing"" = intervalVar(size=5), ""house:2 task:roofing"" = intervalVar(size=5), ""house:3 task:roofing"" = intervalVar(size=5), ""house:0 task:facade"" = intervalVar(size=10), ""house:1 task:facade"" = intervalVar(size=10), ""house:2 task:facade"" = intervalVar(size=10), ""house:3 task:facade"" = intervalVar(size=10), ""house:0 task:garden"" = intervalVar(size=5), ""house:1 task:garden"" = intervalVar(size=5), ""house:2 task:garden"" = intervalVar(size=5), ""house:3 task:garden"" = intervalVar(size=5)]"


In [60]:
df = pd.merge( house_tasks, Workers, on = ['TaskName'])
Worker_tasks = df[['HouseName','WorkerName','TaskDV']].groupby('WorkerName').agg(list).reset_index()


Worker_tasks ['WorkerDV']=0
for index, row in Worker_tasks.iterrows():
    print(row.WorkerName)
    dv = mdl.sequence_var ( row.TaskDV, types = row.HouseName,  name="worker:{}".format(row.WorkerName))
    Worker_tasks.at[index,'WorkerDV']=dv

display(Worker_tasks.head(2))

Jim
Joe


,WorkerName,HouseName,TaskDV,WorkerDV
0,Jim,"[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]","[""house:0 task:plumbing"" = intervalVar(size=40), ""house:1 task:plumbing"" = intervalVar(size=40), ""house:2 task:plumbing"" = intervalVar(size=40), ""house:3 task:plumbing"" = intervalVar(size=40), ""house:0 task:ceiling"" = intervalVar(size=15), ""house:1 task:ceiling"" = intervalVar(size=15), ""house:2 task:ceiling"" = intervalVar(size=15), ""house:3 task:ceiling"" = intervalVar(size=15), ""house:0 task:painting"" = intervalVar(size=10), ""house:1 task:painting"" = intervalVar(size=10), ""house:2 task:painting"" = intervalVar(size=10), ""house:3 task:painting"" = intervalVar(size=10), ""house:0 task:windows"" = intervalVar(size=5), ""house:1 task:windows"" = intervalVar(size=5), ""house:2 task:windows"" = intervalVar(size=5), ""house:3 task:windows"" = intervalVar(size=5), ""house:0 task:moving"" = intervalVar(size=5), ""house:1 task:moving"" = intervalVar(size=5), ""house:2 task:moving"" = intervalVar(size=5), ""house:3 task:moving"" = intervalVar(size=5)]","""worker:Jim"" = sequenceVar([""house:0 task:plumbing"", ""house:1 task:plumbing"", ""house:2 task:plumbing"", ""house:3 task:plumbing"", ""house:0 task:ceiling"", ""house:1 task:ceiling"", ""house:2 task:ceiling"", ""house:3 task:ceiling"", ""house:0 task:painting"", ""house:1 task:painting"", ""house:2 task:painting"", ""house:3 task:painting"", ""house:0 task:windows"", ""house:1 task:windows"", ""house:2 task:windows"", ""house:3 task:windows"", ""house:0 task:moving"", ""house:1 task:moving"", ""house:2 task:moving"", ""house:3 task:moving""], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])"
1,Joe,"[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]","[""house:0 task:masonry"" = intervalVar(size=35), ""house:1 task:masonry"" = intervalVar(size=35), ""house:2 task:masonry"" = intervalVar(size=35), ""house:3 task:masonry"" = intervalVar(size=35), ""house:0 task:carpentry"" = intervalVar(size=15), ""house:1 task:carpentry"" = intervalVar(size=15), ""house:2 task:carpentry"" = intervalVar(size=15), ""house:3 task:carpentry"" = intervalVar(size=15), ""house:0 task:roofing"" = intervalVar(size=5), ""house:1 task:roofing"" = intervalVar(size=5), ""house:2 task:roofing"" = intervalVar(size=5), ""house:3 task:roofing"" = intervalVar(size=5), ""house:0 task:facade"" = intervalVar(size=10), ""house:1 task:facade"" = intervalVar(size=10), ""house:2 task:facade"" = intervalVar(size=10), ""house:3 task:facade"" = intervalVar(size=10), ""house:0 task:garden"" = intervalVar(size=5), ""house:1 task:garden"" = intervalVar(size=5), ""house:2 task:garden"" = intervalVar(size=5), ""house:3 task:garden"" = intervalVar(size=5)]","""worker:Joe"" = sequenceVar([""house:0 task:masonry"", ""house:1 task:masonry"", ""house:2 task:masonry"", ""house:3 task:masonry"", ""house:0 task:carpentry"", ""house:1 task:carpentry"", ""house:2 task:carpentry"", ""house:3 task:carpentry"", ""house:0 task:roofing"", ""house:1 task:roofing"", ""house:2 task:roofing"", ""house:3 task:roofing"", ""house:0 task:facade"", ""house:1 task:facade"", ""house:2 task:facade"", ""house:3 task:facade"", ""house:0 task:garden"", ""house:1 task:garden"", ""house:2 task:garden"", ""house:3 task:garden""], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])"


In [61]:
#Each of the tasks requires a particular worker. As a worker can perform only one task at a time, 
# it is necessary to know all of the tasks that a worker must perform and then constrain that these intervals 
# not overlap and respect the transition times. A sequence variable represents the order in which the workers perform the tasks.


tt = [[int(abs(i - j)) for j in Houses['HouseName'].values] for i in Houses['HouseName'].values]
display(tt)

transitionTimes = transition_matrix(tt)

[[0, 1, 2, 3], [1, 0, 1, 2], [2, 1, 0, 1], [3, 2, 1, 0]]

In [62]:
Worker_tasks

,WorkerName,HouseName,TaskDV,WorkerDV
0,Jim,"[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]","[""house:0 task:plumbing"" = intervalVar(size=40), ""house:1 task:plumbing"" = intervalVar(size=40), ""house:2 task:plumbing"" = intervalVar(size=40), ""house:3 task:plumbing"" = intervalVar(size=40), ""house:0 task:ceiling"" = intervalVar(size=15), ""house:1 task:ceiling"" = intervalVar(size=15), ""house:2 task:ceiling"" = intervalVar(size=15), ""house:3 task:ceiling"" = intervalVar(size=15), ""house:0 task:painting"" = intervalVar(size=10), ""house:1 task:painting"" = intervalVar(size=10), ""house:2 task:painting"" = intervalVar(size=10), ""house:3 task:painting"" = intervalVar(size=10), ""house:0 task:windows"" = intervalVar(size=5), ""house:1 task:windows"" = intervalVar(size=5), ""house:2 task:windows"" = intervalVar(size=5), ""house:3 task:windows"" = intervalVar(size=5), ""house:0 task:moving"" = intervalVar(size=5), ""house:1 task:moving"" = intervalVar(size=5), ""house:2 task:moving"" = intervalVar(size=5), ""house:3 task:moving"" = intervalVar(size=5)]","""worker:Jim"" = sequenceVar([""house:0 task:plumbing"", ""house:1 task:plumbing"", ""house:2 task:plumbing"", ""house:3 task:plumbing"", ""house:0 task:ceiling"", ""house:1 task:ceiling"", ""house:2 task:ceiling"", ""house:3 task:ceiling"", ""house:0 task:painting"", ""house:1 task:painting"", ""house:2 task:painting"", ""house:3 task:painting"", ""house:0 task:windows"", ""house:1 task:windows"", ""house:2 task:windows"", ""house:3 task:windows"", ""house:0 task:moving"", ""house:1 task:moving"", ""house:2 task:moving"", ""house:3 task:moving""], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])"
1,Joe,"[0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]","[""house:0 task:masonry"" = intervalVar(size=35), ""house:1 task:masonry"" = intervalVar(size=35), ""house:2 task:masonry"" = intervalVar(size=35), ""house:3 task:masonry"" = intervalVar(size=35), ""house:0 task:carpentry"" = intervalVar(size=15), ""house:1 task:carpentry"" = intervalVar(size=15), ""house:2 task:carpentry"" = intervalVar(size=15), ""house:3 task:carpentry"" = intervalVar(size=15), ""house:0 task:roofing"" = intervalVar(size=5), ""house:1 task:roofing"" = intervalVar(size=5), ""house:2 task:roofing"" = intervalVar(size=5), ""house:3 task:roofing"" = intervalVar(size=5), ""house:0 task:facade"" = intervalVar(size=10), ""house:1 task:facade"" = intervalVar(size=10), ""house:2 task:facade"" = intervalVar(size=10), ""house:3 task:facade"" = intervalVar(size=10), ""house:0 task:garden"" = intervalVar(size=5), ""house:1 task:garden"" = intervalVar(size=5), ""house:2 task:garden"" = intervalVar(size=5), ""house:3 task:garden"" = intervalVar(size=5)]","""worker:Joe"" = sequenceVar([""house:0 task:masonry"", ""house:1 task:masonry"", ""house:2 task:masonry"", ""house:3 task:masonry"", ""house:0 task:carpentry"", ""house:1 task:carpentry"", ""house:2 task:carpentry"", ""house:3 task:carpentry"", ""house:0 task:roofing"", ""house:1 task:roofing"", ""house:2 task:roofing"", ""house:3 task:roofing"", ""house:0 task:facade"", ""house:1 task:facade"", ""house:2 task:facade"", ""house:3 task:facade"", ""house:0 task:garden"", ""house:1 task:garden"", ""house:2 task:garden"", ""house:3 task:garden""], [0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])"


In [63]:
#no overlap constraint

for index, row in Worker_tasks.iterrows():
    mdl.add( mdl.no_overlap(row.WorkerDV, transitionTimes))

In [64]:
Houses['endDate'] = Houses['HouseDV'].apply(lambda x:mdl.end_of(x))
Houses['Length'] = Houses['HouseDV'].apply(lambda x: mdl.length_of(x))
Houses['distance_To_duedate'] = Houses[['endDate','DueDate']].apply(lambda x:mdl.max( [0, x.endDate - x.DueDate]), axis = 1)
Houses['cost']= Houses['Weight'] * Houses['distance_To_duedate'] + Houses['Length'] 

Houses.head()

#mdl.add_kpi(mdl.sum(Houses['cost']), 'AAA')

,HouseName,ReleaseDate,DueDate,Weight,HouseDV,endDate,Length,distance_To_duedate,cost
0,0,0,120,100.0,"""house:0"" = intervalVar(start=0..1000)","endOf(""house:0"")","lengthOf(""house:0"")","max(0, endOf(""house:0"") - 120)","100.0 * max(0, endOf(""house:0"") - 120) + lengthOf(""house:0"")"
1,1,0,212,100.0,"""house:1"" = intervalVar(start=0..1000)","endOf(""house:1"")","lengthOf(""house:1"")","max(0, endOf(""house:1"") - 212)","100.0 * max(0, endOf(""house:1"") - 212) + lengthOf(""house:1"")"
2,2,151,304,100.0,"""house:2"" = intervalVar(start=151..1000)","endOf(""house:2"")","lengthOf(""house:2"")","max(0, endOf(""house:2"") - 304)","100.0 * max(0, endOf(""house:2"") - 304) + lengthOf(""house:2"")"
3,3,59,181,200.0,"""house:3"" = intervalVar(start=59..1000)","endOf(""house:3"")","lengthOf(""house:3"")","max(0, endOf(""house:3"") - 181)","200.0 * max(0, endOf(""house:3"") - 181) + lengthOf(""house:3"")"


In [65]:
obj = mdl.minimize (  mdl.sum(Houses['cost']) )
mdl.add(obj)
msol2 = mdl.solve(FailLimit=30000, trace_log=False)

msol2.get_objective_values()
#msol2.get_kpis()

(16939,)

In [66]:
mdl.print_information()

Model: 2411260851
 - source file: C:/Users/m_gor/AppData/Local/Temp/ipykernel_14000/2411260851.py
 - modeling time: 5.41 sec
 - number of integer variables:  0
 - number of interval variables: 44
 - number of sequence variables: 2
 - number of state functions:    0
 - number of float variables:    0
 - number of constraints:        63
 - number of root expressions:   63
 - number of expression nodes:   152
 - operations:                   endBeforeStart: 56, endOf: 4, lengthOf: 4, max: 4, minimize: 1, minus: 4, noOverlap: 2, plus: 4, span: 4, sum: 1, times: 4


In [67]:
A = mdl.get_all_variables()
U= []
for i in A:
    try:
        #start = i.start
        #end = i.end
        start =  msol2.get_var_solution(i).get_start()
        end = msol2.get_var_solution(i).get_end()
    except:
        start= "NAN"
        end = 'NAN'
    U.append([i.name, start, end])

pd.DataFrame(U)

,0,1,2
0,house:3,59,194
1,house:2,215,347
2,house:1,155,271
3,house:0,0,156
4,worker:Joe,NAN,NAN
5,house:3 task:garden,138,143
6,house:2 task:garden,312,317
7,house:1 task:garden,251,256
8,house:0 task:garden,107,112
9,house:3 task:facade,143,153
